<h1>
IBM Applied Data Science Capstone Course by Coursera
</h1>

<h3>Week 5 Final Report
Opening a New Shopping Mall in Bangalore, India </h3>

<ul>
<li>Build a dataframe of neighborhoods in Bangalore, India by web scraping the data from Wikipedia page</li>
<li>Get the geographical coordinates of the neighborhoods</li>
<li>Obtain the venue data for the neighborhoods from Foursquare API</li>
<li>Explore and cluster the neighborhoods</li>
<li>Select the best cluster to open a new shopping mall</li>
</ul>

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")


Libraries imported.


<h1>Scrap data from Wikipedia page into a DataFrame</h1>

In [2]:
data = requests.get("https://en.m.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore").text

In [3]:

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [4]:
neighborhoodList = []
for row in soup.find_all("div", class_="mw-parser-output")[0].findAll("a"):
    neighborhoodList.append(row.text)

In [5]:
b_df = pd.DataFrame({"Neighborhood": neighborhoodList[15:]})
b_df['Neighborhood'].replace('', inplace=True)
b_df.head(10)


,Neighborhood
0,Cantonment
1,British Raj
2,Richmond Town
3,Austin Town
4,Murphy Town
5,Fraser Town
6,Cox Town
7,Domlur
8,
9,HAL


In [6]:
b_df.shape


(187, 1)

In [7]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Banaglore, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [8]:

# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in b_df["Neighborhood"].tolist() ]

In [9]:
coords


[[12.975660000000062, 77.60542000000004],
 [12.966180000000065, 77.58690000000007],
 [12.962330000000065, 77.60123000000004],
 [12.963480000000061, 77.61297000000008],
 [12.980030000000056, 77.62776000000008],
 [12.998940000000061, 77.61276000000004],
 [12.995940000000076, 77.62644000000006],
 [12.943290000000047, 77.65602000000007],
 [12.966180000000065, 77.58690000000007],
 [12.960930000000076, 77.66419000000008],
 [13.030060000000049, 77.49526000000003],
 [12.966180000000065, 77.58690000000007],
 [12.906497993046978, 77.5731947496832],
 [13.005440000000021, 77.55693000000008],
 [13.006319846827683, 77.5684048532483],
 [12.966180000000065, 77.58690000000007],
 [12.966180000000065, 77.58690000000007],
 [12.966180000000065, 77.58690000000007],
 [12.975541333026133, 77.52386244169534],
 [12.970150000000046, 77.57589000000007],
 [12.962350000000072, 77.57604000000003],
 [12.96849098279701, 77.57898229462776],
 [13.014830000000075, 77.57771000000008],
 [12.961200000000076, 77.510710000000

In [10]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [11]:
b_df['Latitude'] = df_coords['Latitude']
b_df['Longitude'] = df_coords['Longitude']


In [12]:
print(b_df.shape)
b_df

(187, 3)


,Neighborhood,Latitude,Longitude
0,Cantonment,12.975660,77.605420
1,British Raj,12.966180,77.586900
2,Richmond Town,12.962330,77.601230
3,Austin Town,12.963480,77.612970
4,Murphy Town,12.980030,77.627760
5,Fraser Town,12.998940,77.612760
6,Cox Town,12.995940,77.626440
7,Domlur,12.943290,77.656020
8,,12.966180,77.586900
9,HAL,12.960930,77.664190


In [13]:
b_df.to_csv("b_df.csv", index=False)

Create a map of Bangalore with neighborhoods superimposed on top

In [14]:
# get the coordinates of Kuala Lumpur
address = 'Bangalore,India'

geolocator = Nominatim(user_agent="http")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore, India{}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore, India12.9791198, 77.5912997.


In [15]:

map_b = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to map
for lat, lng, neighborhood in zip(b_df['Latitude'], b_df['Longitude'], b_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_b)  
    
map_b

In [16]:
map_b.save('map_b.html')

In [17]:
CLIENT_ID = 'NZ0DM2ODTFWSE31NHMSMVZCX4A5YZMF4WKKVGGZTNCXVRXEM' # your Foursquare ID
CLIENT_SECRET = 'FDAGFHSOHZ2YTIACFGF2XY0ULG5A0EBLJTJETK3OVSCEOYJV' # your Foursquare Secret
VERSION = '20210609' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NZ0DM2ODTFWSE31NHMSMVZCX4A5YZMF4WKKVGGZTNCXVRXEM
CLIENT_SECRET:FDAGFHSOHZ2YTIACFGF2XY0ULG5A0EBLJTJETK3OVSCEOYJV


In [18]:

radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(b_df['Latitude'], b_df['Longitude'],b_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [19]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(12237, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Cantonment,12.97566,77.60542,M.G Road Boulevard,12.975771,77.603979,Plaza
1,Cantonment,12.97566,77.60542,Blossom Book House,12.975042,77.604813,Bookstore
2,Cantonment,12.97566,77.60542,Hysteria,12.974843,77.605426,Music Store
3,Cantonment,12.97566,77.60542,Coast 2 Coast,12.975305,77.605625,Indian Restaurant
4,Cantonment,12.97566,77.60542,The 13th Floor,12.975364,77.604995,Lounge


In [20]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
,700,700,700,700,700,700
100 Feet Road,79,79,79,79,79,79
Anekal,2,2,2,2,2,2
Anjanapura,7,7,7,7,7,7
Anugraha,24,24,24,24,24,24
Arekere,73,73,73,73,73,73
Attibele,6,6,6,6,6,6
Austin Town,100,100,100,100,100,100
Avenue Road,100,100,100,100,100,100


In [21]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 232 uniques categories.


In [22]:
venues_df['VenueCategory'].unique()[:50]


array(['Plaza', 'Bookstore', 'Music Store', 'Indian Restaurant', 'Lounge',
       'Café', 'Ice Cream Shop', 'Bar', 'American Restaurant',
       'Toy / Game Store', 'Burger Joint', 'Afghan Restaurant',
       'South Indian Restaurant', 'Italian Restaurant', 'Hotel', 'Gym',
       'Brewery', 'Fast Food Restaurant', 'Deli / Bodega',
       'Cricket Ground', 'Sushi Restaurant', 'Park', 'Cocktail Bar',
       'Pub', 'Breakfast Spot', 'Cupcake Shop', 'Gym / Fitness Center',
       'Chinese Restaurant', 'Japanese Restaurant', 'Soccer Stadium',
       'French Restaurant', 'Shopping Mall', 'Dessert Shop',
       'Andhra Restaurant', 'Tea Room', 'Eastern European Restaurant',
       'Asian Restaurant', 'Department Store', 'Hotel Bar',
       'Thai Restaurant', 'Middle Eastern Restaurant', 'Coffee Shop',
       'Korean Restaurant', 'Diner', 'Sandwich Place', 'Steakhouse',
       'Bakery', 'BBQ Joint', 'Bed & Breakfast', 'Arcade'], dtype=object)

In [23]:
"Neighborhood" in venues_df['VenueCategory'].unique()


True

In [24]:
b_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
b_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [b_onehot.columns[-1]] + list(b_onehot.columns[:-1])
b_onehot = b_onehot[fixed_columns]

print(b_onehot.shape)
b_onehot.head()

(12237, 233)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Terminal,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Caribbean Restaurant,Castle,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,City,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Go Kart Track,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Other Great Outdoors,Other Nightlife,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Print Shop,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Rest Area,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tech Startup,Tennis Stadium,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toll Booth,Toll Plaza,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Tree,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store
0,Cantonment,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Cantonment,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [25]:
b_grouped = b_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(b_grouped.shape)
b_grouped

(127, 233)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Terminal,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Capitol Building,Caribbean Restaurant,Castle,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,City,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Go Kart Track,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Other Great Outdoors,Other Nightlife,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Print Shop,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Rest Area,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tech Startup,Tennis Stadium,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toll Booth,Toll Plaza,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Tree,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store
0,,0.000000,0.000000,0.000000,0.00000,0.000000,0.010000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.010000,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.000000,0.000000,0.030000,0.010000,0.000000,0.000000,0.000000,0.010000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.01,0.010000,0.00,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.010000,0.030000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.020000,0.000000,0.010000,0.000000,0.

In [26]:
len(b_grouped[b_grouped["Shopping Mall"] > 0])


42

In [62]:

b_mall = b_grouped[["Neighborhoods","Shopping Mall"]]
b_mall


,Neighborhoods,Shopping Mall
0,,0.010000
1,100 Feet Road,0.000000
2,Anekal,0.000000
3,Anjanapura,0.000000
4,Anugraha,0.000000
5,Arekere,0.013699
6,Attibele,0.000000
7,Austin Town,0.010000
8,Avenue Road,0.020000
9,BTM Layout,0.000000


Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Bangalore into 5 clusters.

In [49]:
# set number of clusters
bclusters = 3


b_clustering = b_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=bclusters, random_state=0).fit(b_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 2, 0, 0, 2, 0])

In [50]:
bmerged = b_mall.copy()

# add clustering labels
bmerged["Cluster Labels"] = kmeans.labels_

In [51]:

bmerged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
bmerged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,,0.01,0
1,100 Feet Road,0.00,0
2,Anekal,0.00,0
3,Anjanapura,0.00,0
4,Anugraha,0.00,0


In [52]:
bmerged = bmerged.join(b_df.set_index("Neighborhood"), on="Neighborhood")

print(bmerged.shape)
bmerged.head() # check the last columns!

(187, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,,0.01,0,12.96618,77.5869
0,,0.01,0,12.96618,77.5869
0,,0.01,0,12.96618,77.5869
0,,0.01,0,12.96618,77.5869
0,,0.01,0,12.96618,77.5869


In [53]:
print(bmerged.shape)
bmerged.sort_values(["Cluster Labels"], inplace=True)
bmerged


(187, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,,0.010000,0,12.966180,77.586900
102,Russell Market,0.010000,0,12.985070,77.605510
103,SH35,0.000000,0,12.905131,77.746758
103,SH35,0.000000,0,12.905131,77.746758
103,SH35,0.000000,0,12.905131,77.746758
104,SH39,0.000000,0,13.104359,77.478086
104,SH39,0.000000,0,13.104359,77.478086
105,SH87,0.000000,0,12.928390,77.600570
106,Sadashivanagar,0.000000,0,13.014830,77.577710
101,Richmond Town,0.010000,0,12.962330,77.601230


In [54]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(bclusters)
ys = [i+x+(i*x)**2 for i in range(bclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bmerged['Latitude'], bmerged['Longitude'], bmerged['Neighborhood'], bmerged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [55]:
map_clusters.save('map_clusters.html')

In [57]:

b_merged.loc[b_merged['Cluster Labels'] == 0]

,Neighborhoods,Shopping Mall,Cluster Labels
63,Kengeri,0.0,0
72,Mahadevapura,0.0,0
70,Madiwala,0.0,0
110,Thavarekere,0.0,0
67,Kumaraswamy Layout,0.0,0
65,Kothnur,0.0,0
64,Koramangala,0.0,0
125,railway station,0.0,0
62,Kempe Gowda I,0.0,0
113,Uttarahalli,0.0,0


In [59]:
b_merged.loc[b_merged['Cluster Labels'] == 1]

,Neighborhoods,Shopping Mall,Cluster Labels
54,Jalahalli,0.2,1


In [61]:
b_merged.loc[b_merged['Cluster Labels'] == 2]


,Neighborhoods,Shopping Mall,Cluster Labels
124,flyover,0.016667,2
8,Avenue Road,0.020000,2
48,Hulimavu,0.017544,2
68,Lingarajapuram,0.015385,2
34,Edit,0.023529,2
29,Chickpet,0.016949,2
45,Hoodi,0.022222,2
71,Madiwala Lake,0.020833,2
11,Banashankari,0.020000,2
73,Mahalakshmi Layout,0.018182,2


Most of the shopping malls are concentrated in the central area of Bangalore south city, with the highest number in cluster 2 and moderate number in cluster 1. On the other hand, cluster 0 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 0 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 1 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of shopping malls and suffering from intense competition.